<a href="https://colab.research.google.com/github/matthewpecsok/data_engineering/blob/main/tutorials/de_queries_indexes_query_plans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
!pip -q install --upgrade ipython
!pip -q install --upgrade ipython-sql

!wget -O northwind.db https://github.com/matthewpecsok/data_engineering/raw/main/data/northwind.db

--2024-03-12 21:30:56--  https://github.com/matthewpecsok/data_engineering/raw/main/data/northwind.db
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/matthewpecsok/data_engineering/main/data/northwind.db [following]
--2024-03-12 21:30:57--  https://raw.githubusercontent.com/matthewpecsok/data_engineering/main/data/northwind.db
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 602112 (588K) [application/octet-stream]
Saving to: ‘northwind.db’

northwind.db        100%[===================>] 588.00K  --.-KB/s    in 0.04s   

2024-03-12 21:30:57 (13.3 MB/s) - ‘northwind.db’ saved [602112/602112]



In [44]:
import sqlite3
import pandas as pd

In [45]:
conn = sqlite3.connect('northwind.db') # Connect to the SQLite database
cur = conn.cursor() # Create a cursor object to northwind

In [46]:
pd.read_sql_query("SELECT * FROM orders", conn)

,OrderID,CustomerID,EmployeeID,OrderDate,RequiredDate,ShippedDate,ShipVia,Freight,ShipName,ShipAddress,ShipCity,ShipRegion,ShipPostalCode,ShipCountry
0,10248,VINET,5,2016-07-04,2016-08-01,2016-07-16,3,32.38,Vins et alcools Chevalier,59 rue de l-Abbaye,Reims,Western Europe,51100,France
1,10249,TOMSP,6,2016-07-05,2016-08-16,2016-07-10,1,11.61,Toms Spezialitäten,Luisenstr. 48,Münster,Western Europe,44087,Germany
2,10250,HANAR,4,2016-07-08,2016-08-05,2016-07-12,2,65.83,Hanari Carnes,"Rua do Paço, 67",Rio de Janeiro,South America,05454-876,Brazil
3,10251,VICTE,3,2016-07-08,2016-08-05,2016-07-15,1,41.34,Victuailles en stock,"2, rue du Commerce",Lyon,Western Europe,69004,France
4,10252,SUPRD,4,2016-07-09,2016-08-06,2016-07-11,2,51.30,Suprêmes délices,"Boulevard Tirou, 255",Charleroi,Western Europe,B-6000,Belgium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825,11073,PERIC,2,2018-05-05,2018-06-02,None,2,24.95,Pericles Comidas clásicas,Calle Dr. Jorge Cash 321,México D.F.,Central America,5033,Mexico
826,11074,SIMOB,7,2018-05-06,2018-06-03,None,2,18.44,Simons bistro,Vinbæltet 34,Kobenhavn,Northern Europe,1734,Denmark
827,11075,RICSU,8,2018-05-06,2018-06-03,None,2,6.19,Richter Supermarkt,Starenweg 5,Genève,Western Europe,1204,Switzerland
828,11076,BONAP,4,2018-05-06,2018-06-03,None,2,38.28,Bon app-,"12, rue des Bouchers",Marseille,Western Europe,13008,France


# show the query plan for the orders table when using the Frieght column

In [47]:
cur.execute("EXPLAIN QUERY PLAN SELECT * FROM orders where Freight > 0").fetchall()

[(2, 0, 0, 'SCAN orders')]

# check the master table for the presence of an index. there is none.

In [48]:
cur.execute("SELECT * FROM sqlite_master WHERE type = 'index' AND tbl_name = 'orders';").fetchall()

[]

# create an index on the table.

In [49]:
cur.execute("CREATE INDEX IF NOT EXISTS idx_orders_Freight ON Orders (Freight);")
conn.commit()

# show the query plan for the orders table when using the Frieght column

now, there is an index

In [50]:
cur.execute("SELECT * FROM sqlite_master where tbl_name='Orders' and type='index';").fetchall()

[('index',
  'idx_orders_Freight',
  'Orders',
  148,
  'CREATE INDEX idx_orders_Freight ON Orders (Freight)')]

In [51]:
pd.read_sql_query("SELECT * FROM sqlite_master where tbl_name='Orders';", conn)

,type,name,tbl_name,rootpage,sql
0,table,Orders,Orders,91,CREATE TABLE [Orders](\n [OrderID]INTEGER NO...
1,index,idx_orders_Freight,Orders,148,CREATE INDEX idx_orders_Freight ON Orders (Fre...


Now show the sqlite query plan. The query engine is planning to use the index to retrieve the rows.

In [52]:
cur.execute("EXPLAIN QUERY PLAN SELECT * FROM orders where Freight > 0").fetchall()

[(3, 0, 0, 'SEARCH orders USING INDEX idx_orders_Freight (Freight>?)')]

In [54]:
cur.execute("EXPLAIN QUERY PLAN SELECT * FROM orders where ShipRegion ='Western Europe'").fetchall()

[(2, 0, 0, 'SCAN orders')]